In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter
import pickle

In [2]:
# Load preprocessed data
with open(
    "C:/Users/syafi/Desktop/syafiq-project/new classification task/model/saved_data/preprocessed_data.pkl",
    "rb",
) as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

# Display train and test sizes
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

Train size: 1344719
Test size: 336180


In [3]:
# Define sizes for experiments
sample_sizes = [25, 50, 75, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 10000, 20000, 30000, 40000, 50000]
training_sizes = [20, 40, 60, 80, 200, 400, 600, 800, 2000, 4000, 6000, 8000, 16000, 24000, 32000, 40000]
testing_sizes = [5, 10, 15, 20, 50, 100, 150, 200, 500, 1000, 1500, 2000, 4000, 6000, 8000, 10000]

# Class labels for reporting
class_labels = [
    "Atherosclerosis",
    "Hypertension",
    "Cardiovascular Disease (CVD)",
    "Chronic Fatigue Syndrome (CFS)",
    "Respiratory Disease (COPD or Asthma)",
    "Stress-related Disorders",
    "Arrhythmias",
    "Healthy",
    "Autonomic Dysfunction",
    "Diabetes",
    "Anaemia",
]

In [4]:
# Evaluate the model and return classification metrics
def evaluate_model(model, X_test_subset, y_test_subset, class_labels):
    predictions = model.predict(X_test_subset)
    accuracy = accuracy_score(y_test_subset, predictions)
    
    # Generate dynamic class labels
    unique_classes = np.unique(y_test_subset)
    dynamic_labels = [
        class_labels[i] for i in unique_classes
    ]
    
    # Generate classification report
    report = classification_report(
        y_test_subset,
        predictions, 
        target_names=dynamic_labels,
        labels=unique_classes,
        zero_division=0,
        output_dict=True
    )
    
    return accuracy, report, predictions, unique_classes, dynamic_labels

In [5]:
# Flatten classification report for easier analysis
def flatten_classification_report(report, sample_size, train_size, test_size, accuracy):
    flat_report = {
        f"{label}_{metric}": value 
        for label, metrics in report.items()
        if isinstance(metrics, dict) # Exclude keys like accuracy
        for metric, value in metrics.items()
    }
    flat_report.update(
        {
            "sample_size": sample_size,
            "train_size": train_size,
            "test_size": test_size,
            "accuracy": accuracy,
        }
    )
    
    return flat_report

In [ ]:
"""
def expand_test_set(X_test, y_test, repeat_factor):

    Expand the test set by repeating each sample multiple times.

    Args:
        X_test (ndarray): Original test features.
        y_test (ndarray): Original test labels.
        repeat_factor (int): Number of times to repeat each test sample.

    Returns:
        Expanded test features and labels.

    X_test_expanded = np.repeat(X_test, repeats=repeat_factor, axis=0)
    y_test_expanded = np.repeat(y_test, repeats=repeat_factor)
    return X_test_expanded, y_test_expanded

"""

In [ ]:
# Container for results
results = []
best_accuracy = 0
best_model = None
best_sample_size = 0

# Iterate through sample, training, and testing sizes
for sample_size, train_size, test_size in zip(
    sample_sizes, training_sizes, testing_sizes
):
    # Perform stratified sampling on the training set
    X_train_subset, _, y_train_subset, _ = train_test_split(
        X_train, y_train, train_size=train_size, stratify=y_train, random_state=42
    )

    # Perform stratified sampling on the testing set
    X_test_subset, _, y_test_subset, _ = train_test_split(
        X_test, y_test, test_size=test_size, stratify=y_test, random_state=42
    )

    # Fit the Naive Bayes model
    nb_model = GaussianNB()
    nb_model.fit(X_train_subset, y_train_subset)

    # Evaluate the model
    accuracy, report, predictions, unique_classes, dynamic_labels = evaluate_model(
        nb_model, X_test_subset, y_test_subset, class_labels
    )

    # Print results for the current iteration
    print(f"\nNaive Bayes Sample size {sample_size} - Accuracy: {accuracy:.4f}")
    print(
        classification_report(
            y_test_subset,
            predictions,
            target_names=dynamic_labels,
            labels=unique_classes,
            zero_division=0,
        )
    )

    # Flatten report and store results
    flat_report = flatten_classification_report(
        report, sample_size, train_size, test_size, accuracy
    )
    results.append(flat_report)

    # Update the best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = nb_model
        best_sample_size = sample_size
        print(
            f"New best model found for sample size {sample_size} with accuracy {accuracy:.4f}"
        )

Skipping sample size 25 due to insufficient test_size (5) for all classes
Skipping sample size 50 due to insufficient test_size (10) for all classes
New best model found for sample size 75 with accuracy 0.2667
New best model found for sample size 100 with accuracy 0.5000
New best model found for sample size 250 with accuracy 0.9000
New best model found for sample size 750 with accuracy 0.9333
New best model found for sample size 7500 with accuracy 0.9400
New best model found for sample size 30000 with accuracy 0.9412


In [ ]:
# Save the best model
if best_model is not None:
    best_model_file = f"best_nb_model_sample_size_{best_sample_size}.pkl"
    with open(best_model_file, "wb") as model_file:
        pickle.dump(best_model, model_file)
    print(f"\nBest model saved as {best_model_file} with accuracy {best_accuracy:.4f}")

In [ ]:
# Save results to a CSV file for further analysis
results_df = pd.DataFrame(results)
results_df.to_csv("naive_bayes_results.csv", index=False)
print("\nResults saved to 'naive_bayes_results.csv'")

In [ ]:
# Display train and test set details
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")
print("\nClass Distribution in Test Set:")
print(y_test.value_counts())